# COMS W1002 Computing in Economics  
## Project created by Professor Karl Sigman
## Option Pricing Using Monte Carlo Simulation

__Problem 1:__ For the values of $p=0.5$ and $p=0.75$ obtain an estimate of $p$ by using Monte Carlo Simulation with $n=100, n=1000, n=10,000$ You will generate $n$ _iid_ Bernoulli $(p)$ random variables $B_1, ..., B_n$ and use as the estimate $$ \hat{p}= \frac{1}{n}\sum_{i=1}^{n}B_i $$ 
This is to show you (convince you) how accurate the Monte Carlo method can be, and how the Strong Law of Large Numbers (SLLN) works in practice.

In [7]:
import numpy as np

def monte_carlo_estimate(p, n):
    """Estimate the probability p using Monte Carlo simulation."""
    bernoulli_trials = np.random.binomial(1, p, n)
    p_estimate = np.mean(bernoulli_trials)
    return p_estimate

# Parameters
probabilities = [0.5, 0.75]
n_values = [100, 1000, 10000]

# Simulation
for p in probabilities:
    print(f"\nEstimates for p = {p}:")
    for n in n_values:
        estimate = monte_carlo_estimate(p, n)
        print(f"Estimate with n = {n}: {estimate}")



Estimates for p = 0.5:
Estimate with n = 100: 0.53
Estimate with n = 1000: 0.494
Estimate with n = 10000: 0.4965

Estimates for p = 0.75:
Estimate with n = 100: 0.79
Estimate with n = 1000: 0.769
Estimate with n = 10000: 0.7507


__Problem 2:__ Recall the formula for computing the price $C_0$ of an option (derivative of the BLM stock prices) That yields a payoff at time $T$, denote by $C_T$:
$$ C_0 = \frac{1}{(1+r)^T}E^*(C_T), $$  

where $*$ refers to the fact that we must use the value $p^*$ instead of the original (real) $P$ for the up/down probability of the BLM. (The real value of $P$ is not needed for pricing.) Also recall that for $C_T = (S_T - K)^+$, the European call option, the expected value, $E^*(S_T-K)^+$ can be computed explicitly yielding the famous Black-Scholes-Merton option pricing formula:  
$$ C_0 = \frac{1}{(1+r)^T}\sum_{k=0}^{T} \left( \begin{array}{c}
T \\ k  \end{array} \right )(p^*)^k(1-p^*)^{T-k}(u^k d^{T-k}S_0-K)^+ . $$  

You are to use this formula to exactly obtain the price on the one hand, and then use Monte Carlo simulation on the other hand to compare and thus see how accurate the Monte Carlo method can be. 

Here are the parameters to use: $T = 10, r = 0.05, u = 1.15, d= 1.01, S_0 = 50, K = 70$. Recall that 
$$ p^* = \frac{1+r-d}{u-d} . $$  

For the Monte Carlo, use $n=100, n=1000, n=10,000$ _iid_ copies of $C_T$ (for averaging) to see how it gets more accurate as $n$ increases.

In [9]:
import numpy as np
from scipy.stats import binom

# Given parameters
T, r, u, d, S0, K = 10, 0.05, 1.15, 1.01, 50, 70
p_star = (1 + r - d) / (u - d)

# Black-Scholes-Merton Formula
def black_scholes_merton(T, r, p_star, u, d, S0, K):
    discounted_payoffs = [
        binom.pmf(k, T, p_star) * max(u**k * d**(T-k) * S0 - K, 0)
        for k in range(T+1)
    ]
    C0 = np.exp(-r * T) * sum(discounted_payoffs)
    return C0

# Monte Carlo Simulation
def monte_carlo_simulation(T, r, u, d, S0, K, n):
    payoffs = []
    for _ in range(n):
        ST = S0
        for _ in range(T):
            ST *= u if np.random.rand() < p_star else d
        payoffs.append(max(ST - K, 0))
    C0 = np.exp(-r * T) * np.mean(payoffs)
    return C0

# Calculating option price using Black-Scholes-Merton formula
bsm_price = black_scholes_merton(T, r, p_star, u, d, S0, K)
print(f"Black-Scholes-Merton Price: {bsm_price}")

# Monte Carlo simulation for different n values
for n in [100, 1000, 10000]:
    mc_price = monte_carlo_simulation(T, r, u, d, S0, K, n)
    print(f"Monte Carlo Price with n = {n}: {mc_price}")

Black-Scholes-Merton Price: 7.847876394135765
Monte Carlo Price with n = 100: 7.842380445854078
Monte Carlo Price with n = 1000: 8.092444581732984
Monte Carlo Price with n = 10000: 7.700691244022017


__Problem 3__: With the same parameters as in Problem 2, price the _Asian call option:_ The payoff $C_T$ at atime $T$ is based on the average value of the stock over the $T$ time units:
$$ C_T = \left( \frac{1}{T} \sum_{i=1}^{T} S_i-K \right)^+ $$  

Use $n=100, n=1000, n=10,000$ _iid_ copies of $C_T$ (for averaging).

In [10]:
import numpy as np

# Given parameters
T, r, u, d, S0, K = 10, 0.05, 1.15, 1.01, 50, 70
p_star = (1 + r - d) / (u - d)

# Monte Carlo Simulation for Asian Call Option
def monte_carlo_asian_call(T, r, u, d, S0, K, n):
    payoffs = []
    for _ in range(n):
        stock_prices = [S0]
        # Loop runs T+1 times to generate T intervals (excluding the initial price)
        for _ in range(1, T + 1):
            stock_prices.append(stock_prices[-1] * (u if np.random.rand() < p_star else d))
        # Calculate average excluding the initial stock price S0
        average_stock_price = np.mean(stock_prices[1:])
        payoffs.append(max(average_stock_price - K, 0))
    C0 = np.exp(-r * T) * np.mean(payoffs)
    return C0

# Calculating Asian call option price for different n values
for n in [100, 1000, 10000]:
    asian_call_price = monte_carlo_asian_call(T, r, u, d, S0, K, n)
    print(f"Asian Call Option Price with n = {n}: {asian_call_price}")


Asian Call Option Price with n = 100: 1.3189974755473066
Asian Call Option Price with n = 1000: 1.1443678121825653
Asian Call Option Price with n = 10000: 1.090224142359929


__Problem 4__: With the same parameters as in Problem 2, but the additional parameters $n_1 = 4, n_2 =6,$ and $b=66$: price a _down and out barrier option_, that has payoff at time $T$ of  

$$ C_T = (S_T -K)^+ I \{ S_{n_1} \geq b, S_{n_2} \geq b \}.$$ 

Use $n=100, n=1000, n=10,000$ _iid_ copies of $C_T$ (for averaging).

In the above, recall that for any event $A$, $I\{A\}$ denotes the _indicator_ random variable defined by  
$$ I\{A\} = \left\{ \begin{array}{ll} 1 & \mbox{if $A$ occurs,} \\ 0 & \mbox{if $A$ does not occur.} \end{array} \right. $$

  
Here, $A = \{S_{n_1} \geq b, S_{n_2} \geq b\}$.


In [5]:
import numpy as np

# Given parameters
T, r, u, d, S0, K, n1, n2, b = 10, 0.05, 1.15, 1.01, 50, 70, 4, 6, 66

# Monte Carlo Simulation for Down-and-Out Barrier Option
def monte_carlo_barrier_option(T, r, u, d, S0, K, n1, n2, b, n):
    payoffs = []
    for _ in range(n):
        stock_prices = [S0]
        for t in range(1, T + 1):
            stock_prices.append(stock_prices[-1] * (u if np.random.rand() < p_star else d))

        # Check if the barrier condition is met
        if stock_prices[n1] >= b and stock_prices[n2] >= b:
            payoff = max(stock_prices[-1] - K, 0)
        else:
            payoff = 0
        payoffs.append(payoff)

    C0 = np.exp(-r * T) * np.mean(payoffs)
    return C0

# Calculating barrier option price for different n values
for n in [100, 1000, 10000]:
    barrier_option_price = monte_carlo_barrier_option(T, r, u, d, S0, K, n1, n2, b, n)
    print(f"Down-and-Out Barrier Option Price with n = {n}: {barrier_option_price}")

Down-and-Out Barrier Option Price with n = 100: 5.495249197284891
Down-and-Out Barrier Option Price with n = 1000: 4.5830537386624615
Down-and-Out Barrier Option Price with n = 10000: 4.559527938286149


Sources: ChatGPT